In [ ]:
from __future__ import division, print_function
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist
import scipy as sp
import pandas as pd
import csv
from scipy.spatial.distance import pdist
from sklearn.decomposition import PCA
from mahalanobis import Mahalanobis
from functools import partial
from tqdm import trange
from numba import njit

Loading the dataset and doing PCA dimensional reduction

In [ ]:
#######
dados_crus = pd.read_csv('OutPut5Corr.csv',sep = ';', header=0)
X_reduced = pd.DataFrame(dados_crus.iloc[:, [2,3,4,20,21]])
pca = PCA(n_components=4)
X_reduced = pca.fit_transform(X_reduced)
COV = pd.DataFrame(X_reduced).cov()
ICOV = np.linalg.inv(COV)



In [ ]:
X_reduced.shape

To find the maximum distance we can use the following intuitive loops but it will last years. You can check it and see the remaining time in the indicator

In [ ]:
"""

Dmin = 0.
InitSamples = []
for i in trange(len(X_reduced)):
    for j in range(i, len(X_reduced)):
        Dmax = sp.spatial.distance.mahalanobis(X_reduced[i],X_reduced[j],ICOV)
        if Dmax > Dmin:
            InitSamples = [i,j]
            Dmin = Dmax

"""                

The idea is using Numba. We can define the following function and use `njit` as decorator. But before defining this function for both loops, to see the results, we define a function for just one of the loops.

In [ ]:
@njit
def find_min_check(X, i, VI):
    Dmax = 0.
    #for i in range(len(X)):
    for j in range(i, len(X)):
        D = np.sqrt(np.dot(np.dot((X[i]-X[j]),VI),(X[i]-X[j]).T))
        if D > Dmax:
            InitSamples = [i,j]
            Dmax = D
    return Dmax, InitSamples

In [ ]:
Dmax = 0
Init = 0
for i in trange(len(X_reduced)):
    D, InitSamples = find_min_check(X_reduced, i, ICOV)
    if D > Dmax:
        Init = InitSamples
        Dmax = D

As it can be seen the result is very promising. Now we can put both of the loops in the function. But, unfornunatly, we can not measure the remaining time due to the restrictions of Numba. Hopefully the computation will terminate in a reasonable time!

In [ ]:
@njit
def find_min(X, VI):
    Dmax = 0.
    for i in range(len(X)):
        for j in range(i, len(X)):
            D = np.sqrt(np.dot(np.dot((X[i]-X[j]),VI),(X[i]-X[j]).T))
            if Dmax > D:
                InitSamples = [i,j]
                Dmax = D
    return Dmax, InitSamples
                

In [ ]:
Dmax, InitSamples = find_min(X_reduced, ICOV)

In [ ]:

                
test_size = 0.01               
train_size = round(len(X_reduced) * (1-test_size))

Train_Samples = []
Train_Samples.append(InitSamples[0])
Train_Samples.append(InitSamples[1])

for i in range(len(X_reduced)):
    Test_Samples.append(i)
    
Test_Samples.remove(InitSamples[0])
Test_Samples.remove(InitSamples[1])

for u in range(train_size - 2):
    for h in Train_Samples:
        for j in range(len(X_reduced)):
            if h > j and h != j: #Só posso escrever abaixo assim por ser simétrica//Python lê a memória em colunas ou em linhas?
                DistH = sp.spatial.distance.mahalanobis(X_reduced[h,:],X_reduced[j,:],ICOV)
                if DistH < DminH:
                    DminH = DistH
                    Sample_J_And_MinDist_For_Sample_H = [j,DistH]
        for g in range(len(Train_Samples)):
            Sample_J_And_MinDist_For_Sample_H_Matrix.append(Sample_J_And_MinDist_For_Sample_H)
            
                        
        To_Remove = np.array(Sample_J_And_MinDist_For_Sample_H_Matrix)        
        ArgMaxMinD = np.argmax(To_Remove[:,1])
        
        Train_Samples.append(To_Remove[ArgMaxMinD,0])
        Test_Samples.remove(To_Remove[ArgMaxMinD,0])
        
        Sample_J_And_MinDist_For_Sample_H=[]

with open('Test_Samples.csv', 'a', newline='') as outfile:
    w = csv.writer(outfile, delimiter=';')
    w.writerow(['Test_Samples'])
    w.writerows(Test_Samples.T)

InitSamples
Test_Samples